# Cassandra


### Glossary

Replica: Node storing _a_ value for a key

__Replication:__
1. Simple
  1. Random: Just like Chord, use a hash to figure out where each key goes
  2. ByteOrdered: Assign a range of keys to each server
      1. Good for when you do a lot of ranged lookups (give me all users a-k)
1. NetworkTopology (Multi-Datacenter deployments)
  1. Two/Three replicas per DC
  2. Per DC:
      1. First replica gets dropped by Partitioner
      1. Search around the ring until you get a server on a different rack

__Snitchin'__
* Informs on network topology
* A few options:
    * Simple: Rack-unaware
    * RackInferring: Assumes IP addresses have rack info:
        * `x.<DC>.<rack>.<node>`
    * PropertyFile: Use a config file, holy shit
    * EC2Snitch
        * EC2 Region = DC
        * Availability zone = rack

__Readin' & Writin'__

* Can't lock on writes
* Write path:
    * Client tries to write to a Cassandra Coordinator (some node in the ring)
    * Coordinator uses the Partitioner to output query to all need-to-know nodes in teh ring
    * Coordinator responds to client once _X_ nodes respond
    
__Fault tolerance__
* Coordinator buffers writes if any replica is down (doesn't respond)
* ^This is called "hinted handoff"
* If you've got a multi-DC setup, you might:
    * Elect a per-DC Coordinator (distinct from a Coordinator who talks to clients)
    * Who is the DC Coordinator? They get elected by Zookeeper
        * Zookeeper (I knew you'd ask) is running a consensus-algo called Paxos

__So you're a replica node__
* And you just got a write
* You will
    1. Log the write in a disk commit log
    1. Update a "memtable"
        1. memtable = in-memory repr of multiple k-v pairs
        1. Key-indexed cache
        1. Is Write-back, not write-through
            1. Write-back: Stored in memory
            1. Write-through: Store in mem & disk
        1. Gets flushed to disk periodically
    1. But you also need to store the file on disk anyway, so:
        1. You'll have a data file: List of key-value pairs, sorted by key
            1. A "__String Sorted Table (SSTable)__"
        1. An _index file_: List of (key, position in data file)
        1. And a Bloom Filter in front of that.
* A what? Bloom Filter?
    * Compact way repr'ing a set of items
    * Existence checks become very cheap
        * Has a risk of false positives
        * But never false negatives
    * Bascially a bitmap
        * Initially all values are zero (no keys present)
        * For each key, apply a set of hash algos
            * Set all output bits in the bitmap to 1
        * As you add keys, you start to have over-lapping 1 bits
        * This is where your risk of false positives comes from
        * But it's still pretty low:
            * If you use k=4 hash functions
            * A 3200 bitmap
            * And insert a 100 items,
            * You only have a 0.02% False Positive rate

__Compaction__
* Each server ends up having many SSTables
* A single key may end up in several SSTables
* This is wasteful
* Periodically, merge key entries across SSTables into one.
    * Use most recent key entry
    

__Deletes__
* Don't delete anything immediately
* Instead, mark for deletion with a _tombstone_
* Compaction deletes tombstoned entries

__Readin'__
* Coordinator asks X replicas
* Favors quick-responding replicas from past data
* Then returns the latest-timestamped value (Replicas may not be consistent)

__Consistency__
* Coordinator does background checks for older values
    * Checks replicas for keys
    * Updates older replicas with most recent key information
* What if compaction takes too long? 
    * You can get columns for a key separated across multiple SSTalbles
        * Say you just update one column of the key
    * Reads start to hit multiple SSTables
    * Slow-er, but still fast

## Membership

Any server in the cluster could act as a Coordinator.

Every server needs to know about every other server

Cassandra uses Gossip-style membership.
1. Nodes periodically gossip their membership list
1. On receipt, local membership lists get updated
    1. If any node in the membership list has a heartbeat that's older that `time_to_fail`, then mark the node as failed.
    
### Suspicion Mechanism
* Here "suspicion" means figuring out if a node has failed
* Having a mechanism lets us set these suspect/fail timeouts adaptively.
* Cassandra uses an "Accrual Detector"
    * The Accrual Detector outpus a value PHI
    * PHI represents the probability a node failed
    * PHI takes into account how long messages have historically taken to be received from the suspected node
        * PHI(t) = -log(P(t_now - t_last) / log 10
    * Apps set a PHI threshold instead of hard & fast timeouts. 

## X: The Cap Theorem

Basically, you can have 2 out of these 3:
1. __C__onsistency. All nodes see the same data OR all reads return the latest data
1. __A__vailability. All operations are always avilable, and quickly
1. __P__artition Tolerance. Ability of system to continue normal functioning if you remove a piece of the system.

Since Partition Tolerance is an absolute requirement, you see __Cassandra__ choosing availability, with traiditonal __RDBMS__ valuing consistency but offering poor parition tolerance.

### Eventual Consistency
* If you stop writing to a key, all of its value (replicas) will eventually be the same
* In actual systems with continual writes, you have a lagging "wave" of updated values being pushed to all servers
* If you can have low-write periods, then the system will converge quickly to a consistent state.

__Consistency Levels__
* Cassandra lets you choose how consistent each operation can be
* ALL: First server that writes and returns is accepted. If replicas are down, just cache the write on the Coordinator and signal success back to the client. 
* ANY: Wait for each replica to ACK receipt of write. 
    * End up waiting for slowest replica
    
Next, you have a spectrum of _how many_ replicas must ACK

* ONE: Wait for at least one replica to ACK write receipt
* QUORUM: At least 50% of replicas ACK write receipt
    * Faster than all
    * Pretty strong consistent

#### QUORUM 

__Reads__
* Client specifies R replicas to receive values from
* Coordinator waits for R replicas to respond before itself responding
* Coordinator initiates background read-repair for remaining N-R replicas.

__Writes__
* Client specifies W, number of replicas to receive value
* Coordinator can:
    1. Block until W replicas reply
    1. ACK immediately back to the client, and follow-up with making sure W replicas have been written to.
    
__Ensuring Strong Consistency__
1. W + R > N
    * Make sure at least one server is shared by both read & write quorums
1. W > N/2
    * Greater than 50% of replicas are used for each write
        * Detects conflicts
        * Ensures replication

Cassandra lets you choose several level of QUORUM:
1. QUORUM: Quorum across all replicas in all DCs
1. LOCAL_QUORUM: Quorum in coordinator's DC
1. EACH_QUORUM: Let each DC find its own QUORUM, reply up a hierarchy of potential QUORUMs before reaching Coordinator.

## The Consistency Spectrum

Consistency is on a spectrum:
```
Faster R/W <---------------------------------------- Slower R/W
Eventual <===========================================> Strong
```
There are many models that have been derived upon this spectrum

### Eventual Consistency Models

__Per-key Sequential__ 
* On a per-key basis, all ops have a global order
* maintain a per-key Coord., serialize all ops through that Coord.

__CRDT__
* Commutative Replicated Data Type
* Storing data in a way where you can reverse the order of two ops and get the same result
* Imagine your value is an int, and your op(s) are just +1

__Red-Blue Consistency__
* _Blue_ ops can be executed in any order across DCs
    * Commutative
* _Red_ ops must be executed in the same order.
    * Serial

__Causal__
* Newer notion out of CMU
* "Reads must respet partial order based on info flow"
* Reads must respect causality
* Reads must return most recently-written value

### Strong Consistency Models

__Linearizability__
* Each op by a client is visible/available instantaneously to all other clients

__Sequential__
* Any execution results is the same as if the ops were executed in a sequential order
* Ops can be re-ordered after-the-fact to facilitate a consistent view across all clients

## HBase

* Google's BigTable system was "blob-based"
* Yahoo! open-sourced it to HBase
* __Features__:
    * Get/Put (row)
    * Scan(row range, filter)
    * MultiPut
* Prefers consistency over availability

![HBase Architecture](img/HBase_architecture.png)

### HBase Components

* Client
    * Sends R/W
* Zookeeper
    * Coordinates client & server communication
    
* HBaseTable
    * Table of information
    * Gets split into different regions for _replication_
* ColumnFamily
    * Subset of colunns inside HBaseTable
    * All regions of an HBaseTable contain the same set of columns between their ColumnFamilies
* HRegionServer
    * Contains HRegions: Paritions of an HBase Table
        * Store
            * One Store per combination of ColumnFamily + region 
            * MemTable
                * In-memory key-values
                * Flushed to disk once full
            * StoreFiles
                * Where data lives on disk
            * HFile
                * File stored in Hadoop Distributed File System
                
![HFile Structure](img/HFile.png)

### Write-Ahead Log

![HBase Write-Ahead Log](img/hbase_write_ahead_log.png)

1. Client writes four keys to HRegionServer
1. HRegionServer directs (k1, k2) to HRegion1, (k3, k4) to HRegion2
1. HRegion# servers write to HLog _before_ writing to Store::MemStore

__On failure__
1. HMaster/HRegionServer: Replay HLog
    1. Add edits to appropriate HRegion MemStores
        * These will get flushed to HFiles once full
        
### Master/Slave

* Single Master cluster
* Slave clusters replicate Master's tables
* Master sends HLogs to slave clusters
* Zookeeper used to store/control stateful information
    * Zookeepers presents URLs for storing/retrieving info
        * Example: /hbase/replication/rs/<log id>
    * Service discovery / key-value store